# Evaluating the performance of the CENTRE.MI.MSI consensus LcL classifier accross CT

In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np # calculate the mean and standard deviation
import xgboost as xgb # XGBoost stuff
from xgboost import plot_importance
from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
#from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedKFold, cross_val_score
from numpy import sort

In [2]:
def optimalparamsearch(com):
    com['distance'] =com['distance'].abs()
    com=com.sort_values('pair19')
    com=com.reset_index(drop=True)
    ''' function to do parameter search '''
    cv_names=com["CV"].unique()
    myCViterator = []
    for i in range(len(cv_names)):
        trainIndices = com[ com['CV']!=cv_names[i] ].index.values.astype(int)
        testIndices =  com[ com['CV']==cv_names[i] ].index.values.astype(int)
        myCViterator.append( (trainIndices, testIndices) )


    #run randomized search for optimal parameters

    X_train= com.drop(['gene_id1','gene_id','symbol38','symbol19','pair','pair19','label','CV'], axis=1).copy()
    y_train = com['label'].copy()
    model = xgb.XGBClassifier(objective = "binary:logistic",scale_pos_weight=5,random_state=0)
    param_grid = {
            'max_depth': [4, 5, 6,8,10,12],
            'learning_rate': [0.1, 0.05, 0.01],
            'gamma': [0, 0.25, 1.0],
            'reg_lambda': [0, 1.0, 10.0],
            'n_estimators': [100,200,300,400,500],
            'colsample_bytree': [0.5,0.6,0.7,0.9],
            'subsample': [0.7, 0.9]
        }
    search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,scoring='f1', cv=myCViterator, n_jobs=12, refit=True)
    result = search.fit(X_train, y_train)
    print('est=%.3f, cfg=%s' % (result.best_score_, result.best_params_))
    return(result.best_score_, result.best_params_)

In [3]:
def CVF1results(com, best_params, test):

    cv_names=com["CV"].unique()
    d = dict(tuple(com.groupby('CV')))
    dtest = dict(tuple(test.groupby('CV')))
    result_all_xgboost={}
    result_all_tf={}
    result_cv_xg=[]
    result_cv_tf=[]
    for i in range(len(cv_names)):
        cv_test=cv_names[i]
        print(cv_test)
        cv_train=[x for k,x in enumerate(cv_names) if k!=i]
        print(cv_train)
        X_test_xg = dtest[cv_test].drop(['gene_id1', 'gene_id', 'symbol38', 'symbol19', 'pair', 'pair19', 'label', 'CV'], axis=1).copy()
        y_test_xg = dtest[cv_test]['label'].copy()
        train_xg=pd.concat({k: d[k] for k in cv_train})
        X_train_xg= train_xg.drop(['gene_id1', 'gene_id', 'symbol38', 'symbol19', 'pair', 'pair19', 'label', 'CV'], axis=1).copy()
        y_train_xg = train_xg['label'].copy()
        clf_xgb = xgb.XGBClassifier(objective = "binary:logistic",scale_pos_weight=5,random_state=0,**best_params)
        clf_xgb.fit(X_train_xg, y_train_xg)
        pred_s = clf_xgb.predict_proba(X_test_xg)
        lr_probs =pred_s[:, 1]
        yhat = clf_xgb.predict(X_test_xg)
        result_cv = pd.DataFrame({'pred_prob':lr_probs,'pred_label': yhat,'true_label':y_test_xg})
        result_cv_xg.append(f1_score(result_cv['true_label'], result_cv['pred_label']))
        result_all_xgboost[cv_test]=result_cv
        



    results_xg=pd.concat(result_all_xgboost)
    
    lr_precision_xg, lr_recall_xg, _ = precision_recall_curve(results_xg['true_label'], results_xg['pred_prob'])
    lr_f1_xg, lr_auc_xg = f1_score(results_xg['true_label'], results_xg['pred_label']), auc(lr_recall_xg, lr_precision_xg)
    print('xgboost HiC 12 fold CV:auc=%.3f' % lr_auc_xg)
    print(lr_f1_xg)
    print(result_cv_xg)

    dist_precision, dist_recall, _=precision_recall_curve(com['label'],1/abs(com['distance']))
    dist_auc = auc(dist_recall, dist_precision)
    print('Distance:auc=%.3f' % (dist_auc))
    return(lr_f1_xg, result_cv_xg)

In [8]:
#### Load the CENTRE.MI.MSI classifier
CENTREMIMSI = xgb.Booster()
CENTREMIMSI.load_model("/home/lopez_s/CRUP_scores/CENTRE_HiC/Training/CENTRE_HiC_classifiers/consensusLcL10kb_model.txt")

In [6]:
##General paths and suffixes
suffixMIMSI = "-Benchmark.MI.MSI.v38.csv"
suffixBENGI = "-Benchmark.v38.txt"
rootMIMSI = "/project/CRUP_scores/CENTRE_HiC/Training/BENGI_MSI_MI_datasets/"
rootBENGI = "/project/CRUP_scores/toSara/BENGI_processed_datasets/"

In [7]:
sample = "consensusLcL"
consensusLcLMSIMI = pd.read_csv(rootMIMSI+sample+suffixMIMSI, 
                 header=0, sep=',')

FileNotFoundError: [Errno 2] No such file or directory: '/project/CRUP_scores/CENTRE_HiC/Training/BENGI_MSI_MI_datasets/consensusLcL-Benchmark.MI.MSI.v38.csv'

In [11]:
print(list(consensusLcLMSIMI))

['pair', 'gene_id1', 'gene_id', 'symbol38', 'symbol19', 'pair19', 'label', 'CV', 'EP_prob_enh.1', 'EP_prob_enh.2', 'EP_prob_enh.3', 'EP_prob_enh.4', 'EP_prob_enh.5', 'EP_prob_gene.1', 'EP_prob_gene.2', 'EP_prob_gene.3', 'EP_prob_gene.4', 'EP_prob_gene.5', 'PP_prob_enh.1', 'PP_prob_enh.2', 'PP_prob_enh.3', 'PP_prob_enh.4', 'PP_prob_enh.5', 'PP_prob_gene.1', 'PP_prob_gene.2', 'PP_prob_gene.3', 'PP_prob_gene.4', 'PP_prob_gene.5', 'distance', 'cor_CRUP', 'combined_tests', 'reg_dist_enh', 'norm_reg_dist_enh', 'reg_dist_prom', 'norm_reg_dist_prom', 'RNA_seq', 'min_insulation', 'mean_switch_intensity']


In [4]:
rootMIMSI = "/project/CRUP_scores/CENTRE_HiC/Training/BENGI_MSI_MI_datasets/10Kb"
sample = "consensusLcL"
consensusLcLMSIMI = pd.read_csv("/project/CRUP_scores/CENTRE_HiC/Training/BENGI_MSI_MI_datasets/"+sample+".MI.MSI.v38.10kb30kb.csv", 
                 header=0, sep=',')

consensusLcLMSIMI = consensusLcLMSIMI.drop_duplicates('pair19')

consensusLcL = pd.read_csv("/project/CRUP_scores/CENTRE_HiC/Training/CENTRE_final_training/consensus_LCLs.txt", 
                 header=0, sep='\t')

##Parameter search
com=consensusLcLMSIMI.fillna(0)
com1=consensusLcL.fillna(0)
best_params = {'subsample': 0.9, 'reg_lambda': 0, 'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1, 'gamma': 1.0, 'colsample_bytree': 0.9}
#best_score1, best_params1 = optimalparamsearch(com1)
best_params1 = {"colsample_bytree": 0.7, 
                "gamma" : 1.0, "learning_rate" : 0.1,
                "max_depth" : 5, "n_estimators" : 300, "reg_lambda" : 0, "subsample" : 0.9}

In [8]:
samplelist = ["Colon.GTEx", "GM12878.CHiC", "GM12878.CTCF-ChIAPET", "GM12878.GEUVADIS", "GM12878.HiC", "GM12878.RNAPII-ChIAPET",
                  "IMR90.HiC", "K562.CRISPR", "K562.HiC", "Ovary.GTEx", "Pancreas.GTEx", "Stomach.GTEx"]
f1data = pd.DataFrame(columns = ["Sample", "F1_Score_MIMSI"])
rootMIMSI = "/project/CRUP_scores/CENTRE_HiC/Training/BENGI_MSI_MI_datasets/10Kb/"
for sample in samplelist:
    print(sample)
    MSIMI = pd.read_csv(rootMIMSI+sample+suffixMIMSI, 
                 header=0, sep=',')
    test=MSIMI.fillna(0)
    test['distance'] =test['distance'].abs()
    #run randomized search for optimal parameters
    print("The dataset has the correct column order", list(test) == list(com))
    lr_f1, resultcv  = CVF1results(com, best_params, test)
    f1data.loc[len(f1data)] = [sample, lr_f1]
    print(f1data)


Colon.GTEx
The dataset has the correct column order True
cv-11
['cv-0', 'cv-5', 'cv-10', 'cv-7', 'cv-2', 'cv-1', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-0
['cv-11', 'cv-5', 'cv-10', 'cv-7', 'cv-2', 'cv-1', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-5
['cv-11', 'cv-0', 'cv-10', 'cv-7', 'cv-2', 'cv-1', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-10
['cv-11', 'cv-0', 'cv-5', 'cv-7', 'cv-2', 'cv-1', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-7
['cv-11', 'cv-0', 'cv-5', 'cv-10', 'cv-2', 'cv-1', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-2
['cv-11', 'cv-0', 'cv-5', 'cv-10', 'cv-7', 'cv-1', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-1
['cv-11', 'cv-0', 'cv-5', 'cv-10', 'cv-7', 'cv-2', 'cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-9
['cv-11', 'cv-0', 'cv-5', 'cv-10', 'cv-7', 'cv-2', 'cv-1', 'cv-8', 'cv-3', 'cv-4', 'cv-6']
cv-8
['cv-11', 'cv-0', 'cv-5', 'cv-10', 'cv-7', 'cv-2', 'cv-1', 'cv-9', 'cv-3', 'cv-4', 'cv-6']
cv-3
['cv-11', 'cv-0', 'cv-5', 'cv-10', 'cv-7', 'cv-2', 'cv-1', 'cv-9', 'cv-8',

In [9]:
f1data.to_csv('/project/CRUP_scores/CENTRE_HiC/Training/CENTRE_final_training/f1_consensusLcL.csv')

In [10]:
samplelist = ["Colon.GTEx", "IMR90.HiC", "Ovary.GTEx", "Pancreas.GTEx", "Stomach.GTEx"]
f1data = []
rootMIMSI = "/project/CRUP_scores/CENTRE_HiC/Training/BENGI_MSI_MI_datasets/10Kb/"
for sample in samplelist:
    print(sample)
    MSIMI = pd.read_csv(rootBENGI+sample+suffixBENGI, 
                 header=0, sep='\t')
    test=MSIMI.fillna(0)
    test['distance'] =test['distance'].abs()
    #run randomized search for optimal parameters
    print("The dataset has the correct column order", list(test) == list(com1))
    lr_f1, resultcv  = CVF1results(com1, best_params1, test)


Colon.GTEx
The dataset has the correct column order True
cv-9
['cv-8', 'cv-2', 'cv-3', 'cv-4', 'cv-5', 'cv-7', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-8
['cv-9', 'cv-2', 'cv-3', 'cv-4', 'cv-5', 'cv-7', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-2
['cv-9', 'cv-8', 'cv-3', 'cv-4', 'cv-5', 'cv-7', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-3
['cv-9', 'cv-8', 'cv-2', 'cv-4', 'cv-5', 'cv-7', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-4
['cv-9', 'cv-8', 'cv-2', 'cv-3', 'cv-5', 'cv-7', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-5
['cv-9', 'cv-8', 'cv-2', 'cv-3', 'cv-4', 'cv-7', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-7
['cv-9', 'cv-8', 'cv-2', 'cv-3', 'cv-4', 'cv-5', 'cv-1', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-1
['cv-9', 'cv-8', 'cv-2', 'cv-3', 'cv-4', 'cv-5', 'cv-7', 'cv-11', 'cv-0', 'cv-6', 'cv-10']
cv-11
['cv-9', 'cv-8', 'cv-2', 'cv-3', 'cv-4', 'cv-5', 'cv-7', 'cv-1', 'cv-0', 'cv-6', 'cv-10']
cv-0
['cv-9', 'cv-8', 'cv-2', 'cv-3', 'cv-4', 'cv-5', 'cv-7', 'cv-1', 'cv-11', 